In [2]:
from ckks_package.ckks import *

In [3]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Maximal level
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [4]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([ 3.+3.j, -0.-3.j,  3.-1.j, -3.-1.j]),
 array([-2.+3.j,  3.+3.j, -3.+0.j,  1.+1.j])]

In [5]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[6442450944 - 3146225964X^4096 + 22777503750X^8192 + 15531726148X^12288 - 4294967296X^16384 + 3627629016X^20480 - 4555500750X^24576 + 1784612516X^28672 mod(2^700),
 - 2147483648 + 6773854980X^4096 - 15185002500X^8192 + 9098274531X^12288 + 15032385536X^16384 - 481403051X^20480 + 12148002000X^24576 + 6093048256X^28672 mod(2^700)]

In [6]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [7]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 3.00000000e+00+3.j -3.50041329e-10-3.j  3.00000000e+00-1.j
 -3.00000000e+00-1.j]
[-2.+3.00000000e+00j  3.+3.00000000e+00j -3.+5.23661781e-10j
  1.+1.00000000e+00j]


In [8]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [9]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 1.00000000e+00+6.00000000e+00j  3.00000000e+00-7.17143456e-10j
 -2.43973231e-09-1.00000000e+00j -2.00000000e+00+4.64597028e-10j]
[-15.+3.j           9.-9.00000001j  -9.+3.j          -2.-4.j        ]


In [10]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[ 2.99302034e+00+2.9911035j   1.51759777e-03-2.99314678j
  2.99598468e+00-1.00150139j -2.99117505e+00-0.99624687j]
